## Laboratoire 2

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from deeplib.visualization import make_vizualization_autograd

### Graphe computationnel et backprop
Cette section a pour but de vous familiariser avec les notions de graphe computationnel et de backpropagation, plus particulièrement leur implementation PyTorch. Dans le dernier laboratoire, vous avez vu une version haut-niveau de l'entraînement de réseaux de neurones. À l'inverse, ce laboratoire a pour but de vous donner une intuition du fonctionnement interne de PyTorch. Qui sait? Peut-être voudrai vous un jour implémenter vous même votre librairie de graphe de calcul.

#### Tenseurs et Variables
La structure de données de base dans PyTorch est le `Tensor`. Cette structure de données est comparable au `ndarray` numpy. Le package `torch.Tensor` défini des matrices multi-dimentionnelles et les opérations sur celles-ci. Voici quelques exemples: 

In [ ]:
# Création et initialisation à une normale centrée à 0 et de variance 1.
a = torch.Tensor(10,10)
print(a) # Initialement, le tenseur contient du 'garbage'. Il peut même contenir des NaN
a.normal_()
print(a)
print(torch.mean(a))

> **REMARQUE** Dans l'exemple précédent, la méthode `normal_()` se termine par un underscore. Cela signifie que cette méthode fait une mutation du `Tensor`.

In [ ]:
b = torch.Tensor(10,1).fill_(1)
print(b)
print(a.matmul(b))
print(torch.matmul(a,b))

On peut facilement transférer un `Tensor` sur GPU. Les opérations sur ces `Tensor` seront exécutées sur GPU.

In [ ]:
a_gpu = a.cuda()
b_gpu = b.cuda()
print(a_gpu.matmul(b_gpu))
print(a_gpu.matmul(b_gpu).cpu())

In [ ]:
#TODO corrigez cette opération pour multiplier `a` avec `c_gpu` sur cpu
c_gpu = a_gpu.matmul(b_gpu)
print(a.matmul(c_gpu))

PyTorch fournit également l'API Variable dans le package `torch.autograd`. Une Variable peut être initialisée à partir d'un `Tensor`. Les API de `torch.autograd.Variable` et `torch.Tensor` sont presque identiques. Par contre, on ne peut pas faire d'opérations entre des Tensors et des Variables.

In [ ]:
from torch.autograd import Variable

a_var = Variable(a)
b_var = Variable(b)
print(b, b_var)

In [ ]:
print(a_var.matmul(b_var))

In [ ]:
print(a_var.matmul(b))

En règle générale, les opérations in-place, c'est-à-dire les opérations qui font une mutation directe d'un Tensor (et qui se termine par un underscore), ne sont pas disponibles dans l'API de Variable. Pour les autres opérations (par exemple matmul), il n'y a pas de différence entre les API.

In [ ]:
a.uniform_()
a_var.uniform_()

Pourquoi deux API? Variable englobe un Tensor et contient des champs supplémentaires.

In [ ]:
print(type(a_var))
print(type(a_var.data))
print(a_var.grad)

Variable sert à construire les graphe de calcul et faire la backpropagation du gradient. C'est le sujet de la section suivante.

#### Gradient et Backpropagation

Variable provient du package `torch.autograd`. Comme le nom du package l'indique, il est possible d'automatiquement calculer la dérivée de fonctions calculée à partir d'opérations sur les variables. On indique les Variables qu'on veut dériver avec `requires_grad=True` (par défaut à False). Dans l'exemple suivant, lors du calcul de `w` (propagation avant), PyTorch construit dynamiquement un graphe de calcul indiquant les liens de dépendance entre les variables et les opérations, ce qui permet la backpropagation.

> **NOTE** Contrairement à des librairies comme Tensorflow où le graphe de calcul est statique, PyTorch recrée dynamique le graphe de calcul à chaque itération. Cela permet de modifier la structure du graphe dynamiquement avec du code Python. Par contre, cela rend la visualisation du graphe plus difficile. 

- Autograd
- Appeler backprop deux fois (qu'est-ce qui arrive)?
- Volatile
- requires_gradient true et false pour les variables à entraîner vs. les inputs et les variables freezées

In [ ]:
x = Variable(torch.Tensor(3, 3).uniform_(-1, 1))
y = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
w = torch.matmul(x, y) + x + y + z
print(x, y, z, w)

In [ ]:
print(w.grad_fn)

print(x.grad)
print(y.grad)
print(z.grad)
print(w.grad)

make_vizualization_autograd(w)

In [ ]:
w_grad = torch.ones(w.size())
w.backward(w_grad)

In [ ]:
print(x.grad)
print(y.grad)
print(z.grad)
print(w.grad)

##### Questions
- Exécutez deux fois la cellule qui appelle la fonction .backward(). Qu'arrive-t-il? Pourquoi?
- Quelles variables auraient requires_grad=False dans le contexte d'entraînement de réseaux de neurones?
- Dans l'exemple précédent, pourquoi `w` n'a-t-il pas de gradient?

##### TODO exercice
Faites la mise-à-jour des valeurs de y et z et soustrayant $1 \times 10^{-3}$ fois leur gradient

In [ ]:
# TODO

#### Volatile
Un autre *flag* permet d'indiquer à PyTorch de ne pas calculer le gradient d'une variable. Il s'agit du flag **`volatile=True`**. Par contre, son comportement est différent de `requires_grad=False`. Si une variable est volatile, toutes les variables en sortie des opérations dépendant de la variables volatile seront aussi volatile. Dans le premier exemple, w requière un calcul de gradient car au moins une variable (ici y et z) plus haut dans l'arbre de calcul de la backprop le requière également. Au contraire, dans le deuxième exemple, w ne requière pas de calcul de gradient car au moins une variable (ici x) dans plus haut dans l'arbre de calcul de la backprop est volatile. Cela fait que w est volatile également.

In [ ]:
x = Variable(torch.Tensor(3, 3).uniform_(-1, 1))
y = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
w = torch.matmul(x, y) + x + y + z

print(w.requires_grad, w.volatile)

In [ ]:
x = Variable(torch.Tensor(3, 3).uniform_(-1, 1), volatile=True)
y = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
w = torch.matmul(x, y) + x + y + z

print(w.requires_grad, w.volatile)

##### Question
- Dans quel contexte voudrait-on ne calculer aucun gradient d'un graphe de calcul?

### Fonction d'activation
- Avantage de la ReLU sur sigmoid vs. tanh
- Exemple sur le vanishing gradient
- Réduction d'un réseau à plusieur couches sans non-linéarité à un réseau à une seule couche.

#### Visualisation du dataset

In [ ]:
from deeplib.data import SpiralDataset, train_valid_loaders

dataset = SpiralDataset()
points, labels = dataset.to_numpy()
plt.scatter(points[labels==1,0], points[labels==1,1])
plt.scatter(points[labels==0,0], points[labels==0,1])

> **TIPS** Écrire ici comment fonctionnent `__iter__`, `__next__`, `iter()` et `next()` en python. itertools.islice en python

In [ ]:
train_loader, valid_loader = train_valid_loaders(dataset, 8)

for i, (data, label) in enumerate(train_loader):
    print(data, label)
    if i > 10:
        break

In [ ]:
import itertools
for data in itertools.islice(iter(train_loader), 10):
    print(data)

In [ ]:
print(next(iter(train_loader)))

#### Création de modèles

In [ ]:
class RandomModel(torch.nn.Module):
    
    def __init__(self, n_layers):
        super().__init__()
        torch.manual_seed(12345) # Both Tanh model and ReLU model will have the same random weights
        
        self.layers = []
        for i in range(n_layers):
            layer = nn.Linear(7,7)
            layer.weight.data.normal_(0.0, math.sqrt(2 / 7))
            layer.bias.data.fill_(0)
            self.layers.append(layer)
            self.add_module('layer-%d' % i, layer)
        self.output_layer = nn.Linear(7,2)

        self.nonzero_grad_stats = None
        
    
    def forward(self):
        raise NotImplementedError('Defined in children classes')
       
    
    def _forward_output_layer(self, x):
        out = self.output_layer.forward(x)
        out = F.log_softmax(out, dim=1)
        return out
        
    
    def print_weights_grads(self):
        self.nonzero_grad_stats = []
        for i, layer in enumerate(self.layers):
            print("-----\nLayer %d" % i)
            print("Weight:\n%sWeight gradient:\n%s\n" % (str(layer.weight.data), 
                                                         str(layer.weight.grad)))
            if layer.weight.grad is not None:
                nonzero_grad_indices = torch.nonzero(layer.weight.grad.data)
                nonzero_grad = [layer.weight.grad.data[i,j] for (i,j) in nonzero_grad_indices]
                nonzero_grad_mean = np.mean(np.abs(nonzero_grad))
                self.nonzero_grad_stats.append((len(nonzero_grad), nonzero_grad_mean))
                print("Number of nonzero gradient: %f" % len(nonzero_grad))
                print("Nonzero grad mean: %f" % nonzero_grad_mean)
        

        
class RandomReluModel(RandomModel):
    
    def __init__(self, n_layers):
        super().__init__(n_layers)
        
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer.forward(out)
            out = F.relu(out)
        return self._forward_output_layer(out)
        
        
        
class RandomTanhModel(RandomModel):
    
    def __init__(self, n_layers):
        super().__init__(n_layers)
        
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer.forward(out)
            out = F.tanh(out)
        return self._forward_output_layer(out)


In [ ]:
relu_model = RandomReluModel(10)
tanh_model = RandomTanhModel(10)
relu_model.print_weights_grads()
tanh_model.print_weights_grads()

In [ ]:
data_in, data_out = next(iter(train_loader))
relu_output = relu_model.forward(Variable(data_in))
tanh_output = tanh_model.forward(Variable(data_in))
print(data_in)
print("ReLU model ouput:\n", relu_output)
print("tanh model ouput:\n", tanh_output)

#### TODO Exercice
Vérifiez que le réseau retourne bel et bien des probabilités. Identifiez la ligne de code qui transforme des nombres arbitraires en probabilité. Indice: il y a une erreur volontaire dans le code.

#### Analyse du modèle

In [ ]:
loss = torch.nn.NLLLoss()
relu_loss = loss(relu_output, Variable(data_out))
tanh_loss = loss(tanh_output, Variable(data_out))
print(relu_loss, tanh_loss)

In [ ]:
relu_loss.backward()
tanh_loss.backward()
relu_model.print_weights_grads()
tanh_model.print_weights_grads()

In [ ]:
plt.plot(np.arange(len(relu_model.nonzero_grad_stats)), [x[0] for x in relu_model.nonzero_grad_stats])
plt.plot(np.arange(len(tanh_model.nonzero_grad_stats)), [x[0] for x in tanh_model.nonzero_grad_stats])

In [ ]:
fig, axs = plt.subplots(2)
axs[0].plot(np.arange(len(relu_model.nonzero_grad_stats)), [x[1] / x[0] for x in relu_model.nonzero_grad_stats])
axs[0].plot(np.arange(len(tanh_model.nonzero_grad_stats)), [x[1] / x[0] for x in tanh_model.nonzero_grad_stats])
axs[1].plot(np.arange(4), [x[1] / x[0] for x in relu_model.nonzero_grad_stats[:4]])
axs[1].plot(np.arange(4), [x[1] / x[0] for x in tanh_model.nonzero_grad_stats[:4]])
plt.show()

In [ ]:
heatmap = np.zeros((7,7))
for data in train_loader:
    data_in = Variable(data[0])
    relu_model.forward(data_in)
    nonzero_grad_indices = torch.nonzero(relu_model.layers[6].weight.grad.data)
    for (i, j) in nonzero_grad_indices:
        heatmap[i,j] += 1
print(heatmap)

#### Entraînement

In [ ]:
from torch.optim import SGD

n_epoch = 1000
relu_losses = []
tanh_losses = []
relu_optimizer = SGD(relu_model.parameters(), lr=0.001, momentum=0.9, nesterov=True)
tanh_optimizer = SGD(tanh_model.parameters(), lr=0.001, momentum=0.9, nesterov=True)

for epoch in range(n_epoch):
    if epoch % 100 == 0:
        print("================\nEpoch %d done." % epoch)
    relu_epoch_losses = []
    tanh_epoch_losses = []
    for data_in, data_out in map(lambda data: (Variable(data[0]), Variable(data[1])),
                                 train_loader):
        relu_optimizer.zero_grad()
        tanh_optimizer.zero_grad()
        
        relu_loss = loss(relu_model(data_in), data_out)
        tanh_loss = loss(tanh_model(data_in), data_out)
        relu_epoch_losses.append(float(relu_loss))
        tanh_epoch_losses.append(float(tanh_loss))
        
        relu_loss.backward()
        tanh_loss.backward()
        relu_optimizer.step()
        tanh_optimizer.step()
    relu_losses.append(np.mean(np.asarray(relu_epoch_losses)))
    tanh_losses.append(np.mean(np.asarray(tanh_epoch_losses)))

In [ ]:
plt.plot(np.arange(len(relu_losses)), np.asarray(relu_losses))
plt.plot(np.arange(len(tanh_losses)), np.asarray(tanh_losses))

#### Questions
- Observez la distribution du gradient lors de la backprop. Quelles différences y a-t-il entre la backprop à travers ReLU et à travers tanh?
- Est-ce que, pour deux entrées différentes, les mêmes poids ont un gradient élevé?
- Changez le nombre de couches du réseau. Qu'observez-vous?
- Changez la moyenne de la gaussienne des poids lors de l'initilisation. Qu'observez-vous?
- Identifiez un problème avec la tanh. Identifiez un problème avec la ReLU.